# Import Library

In [1]:
!pip install datasets
!pip install rouge_score
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=bbb0b007be83336298de84ce331b7db2f16cec5643ff146b1c3144ae739de3c2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.3 MB/s eta 0:00:00
    

In [2]:
pip install transformers[torch]

In [3]:
import pandas as pd
import os
import tarfile
import joblib
import re
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import transformers
import torch
import torch

from datasets import load_dataset, load_from_disk
from collections import Counter
from google.colab import drive
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import BertTokenizer, EncoderDecoderModel
from transformers import BertTokenizer, BertModel, BertConfig, DataCollatorForSeq2Seq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import trigrams
from nltk import bigrams
from wordcloud import WordCloud


drive.mount('/content/drive')
nltk.download('stopwords')
nltk.download('punkt')

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 500)

stop_words = set(stopwords.words('indonesian'))

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
print(torch.cuda.is_available())

True


# Load Dataset

In [5]:
path = os.getcwd()
project_path = 'drive/MyDrive/IndonesiaAI/TextSummarization/' # Insert your project path
full_project_path = os.path.join(path, project_path)
dataset_folder = 'dataset'
model_folder = 'models'
dataset_path = os.path.join(full_project_path, dataset_folder)
model_path = os.path.join(full_project_path, model_folder)

In [6]:
# with tarfile.open(os.path.join(dataset_path, 'liputan6_data.tar.gz'), "r:gz") as tar:
#     tar.extractall(path=dataset_path)

In [7]:
# df_canonical = load_dataset("id_liputan6", "canonical", data_dir=os.path.join(dataset_path, 'liputan6_data'))

In [8]:
# train_set = df_canonical['train']
# test_set = df_canonical['test']
# val_set = df_canonical['validation']

In [9]:
# df_train = pd.DataFrame(train_set)
# df_test = pd.DataFrame(test_set)
# df_val = pd.DataFrame(val_set)

In [10]:
# Xtreme ID
x_val_id = pd.read_csv(os.path.join(dataset_path, 'xtreme_dev.csv'))
x_test_id = pd.read_csv(os.path.join(dataset_path, 'xtreme_test.csv'))

In [11]:
len(x_test_id), len(x_val_id)

(3862, 4948)

In [12]:
# df_train.to_csv('df_train.csv')
df_train = pd.read_csv(os.path.join(dataset_path, 'df_train.csv'), index_col=0)
df_train.head()

,id,url,clean_article,clean_summary,extractive_summary
0,26408,https://www.liputan6.com/news/read/26408/pbb-siap-membantu-penyelesaian-konflik-ambon,"Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesai...",Konflik Ambon telah berlangsung selama tiga tahun. Partai Bulan Bintang wilayah Maluku siap memb...,"Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesai..."
1,26410,https://www.liputan6.com/news/read/26410/pesta-rakyat-bali-melepas-matahari-2001,"Liputan6.com, Denpasar: Berbeda dengan sebagian besar warga di dunia, masyarakat Bali mempunyai ...","Masyarakat Bali merayakan Tahun Baru dengan tradisi unik dan pesta rakyat, yaitu melepas matahar...",Tradisi tersebut adalah menggelar upacara adat melepas matahari terakhir 2001 dengan pesta rakya...
2,26411,https://www.liputan6.com/news/read/26411/pk-bertekad-menambah-kursi-di-dpr,"Liputan6.com, Jakarta: Partai Keadilan bertekad untuk menambah wakilnya di DPR dengan menargetka...",Partai Keadilan menargetkan tambahan sejuta pemilih pada Pemilu 2004 untuk menambah kursi di DPR...,"Liputan6.com, Jakarta: Partai Keadilan bertekad untuk menambah wakilnya di DPR dengan menargetka..."
3,26412,https://www.liputan6.com/news/read/26412/situasi-bulungan-kembali-normal,"Liputan6.com, Jakarta: Sekitar Rumah Makan Ayam Bulungan Studio Satu, Jakarta Selatan, hingga Se...","Pascaledakan granat di depan Rumah Makan Ayam Bulungan Studio Satu, Bulungan, Jaksel, situasi ke...","Liputan6.com, Jakarta: Sekitar Rumah Makan Ayam Bulungan Studio Satu, Jakarta Selatan, hingga Se..."
4,26413,https://www.liputan6.com/news/read/26413/ratusan-hektare-sawah-dan-tambak-terendam-air,"Liputan6.com, Jambi: Ratusan hektare sawah dikhawatirkan hancur, menyusul banjir yang melanda be...","Bencana Banjir di Jambi, juga mengakibatkan ratusan hektare sawah dan tambak ikan air tawar tere...",Banjir berketinggian air hingga empat meter itu juga mengakibatkan ikan-ikan yang dibudidayakan ...


In [13]:
# load test data and filter the xtreme id
df_test = pd.read_csv(os.path.join(dataset_path, 'df_test.csv'), index_col=0)
df_test = df_test[df_test['id'].isin(list(x_test_id['id']))]
len(df_test)

3862

In [14]:
# load test data and filter the xtreme id
df_val = pd.read_csv(os.path.join(dataset_path, 'df_val.csv'), index_col=0)
df_val = df_val[df_val['id'].isin(list(x_val_id['id']))]
len(df_val)

4948

# Data Inspection

## Check Null Values

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193883 entries, 0 to 193882
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  193883 non-null  int64 
 1   url                 193883 non-null  object
 2   clean_article       193883 non-null  object
 3   clean_summary       193883 non-null  object
 4   extractive_summary  193883 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.9+ MB


In [16]:
df_train.isna().sum()/len(df_train)

id                    0.0
url                   0.0
clean_article         0.0
clean_summary         0.0
extractive_summary    0.0
dtype: float64

## Check Data Duplication

In [17]:
print('Number of rows before duplicates data dropped: ', len(df_train['url']))
print('Number of rows after duplicates data dropped: ',len(df_train['url'].drop_duplicates()))
print('Number of duplicated rows: ' , len(df_train['url']) - len(df_train['url'].drop_duplicates()))

Number of rows before duplicates data dropped:  193883
Number of rows after duplicates data dropped:  193883
Number of duplicated rows:  0


# Data Preparation

In [18]:
def convert_number_format(text):
    text = re.sub(r'(\d)\.\s+(\d)', r'\1\2', text)
    return text

def remove_parentheses(text):
    pattern = r'\([^)]*\)'
    text = re.sub(pattern, "", text)
    return text

removed_text = ['Liputan6.com', 'Liputan6', 'Liputan']
def remove_text(text, removed_text=removed_text):
    for item in removed_text:
        text = text.replace(item, "")
    return text

def remove_punctuation(text):
    punctuation_pattern = r'[^\w\s]'
    cleaned_text = re.sub(punctuation_pattern, ' ', text)
    return cleaned_text

def remove_extra_spaces(text):
    extra_spaces_pattern = r"\s+"
    cleaned_text = re.sub(extra_spaces_pattern, " ", text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

def get_first_word(text):
    words = text.split()
    if words:
        return words[0]
    else:
        return ""

def lowercase(text):
    lowercase_text = text.lower()
    return lowercase_text

def remove_stopwords(text, stopwords=stop_words):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

def remove_single_words(text):
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1]
    return ' '.join(filtered_words)

words_to_remove = ['Jakarta' ,
                   'Surabaya',
                   'Bandung',
                   'Semarang',
                   'Medan',
                   'Makassar',
                   'Yogyakarta',
                   'Denpasar',
                   'Tangerang',
                   'Bogor'
                   ]

def remove_spec_first_word(input_string, words_to_remove=words_to_remove):
    if isinstance(words_to_remove, str):
        words_to_remove = [words_to_remove.lower()]  # Convert to list if a single string is passed

    first_word = input_string.split()[0]
    for word in words_to_remove:
        if word.lower() in first_word.lower():
            return ' '.join(input_string.split()[1:])
    return input_string

In [19]:
def preprocessing(df, article_col, summary_col, article_prep_col, summary_prep_col):
    # Apply preprocessing functions to article column
    df[article_prep_col] = df[article_col].apply(convert_number_format)
    df[article_prep_col] = df[article_prep_col].apply(remove_text)
    df[article_prep_col] = df[article_prep_col].apply(remove_parentheses)
    df[article_prep_col] = df[article_prep_col].apply(remove_punctuation)
    df[article_prep_col] = df[article_prep_col].apply(remove_extra_spaces)
    df[article_prep_col] = df[article_prep_col].apply(lowercase)
    df[article_prep_col] = df[article_prep_col].apply(remove_spec_first_word)
    df[article_prep_col] = df[article_prep_col].apply(remove_single_words)

    # Apply preprocessing functions to summary column
    df[summary_prep_col] = df[summary_col].apply(convert_number_format)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_text)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_parentheses)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_punctuation)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_extra_spaces)
    df[summary_prep_col] = df[summary_prep_col].apply(lowercase)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_spec_first_word)
    df[summary_prep_col] = df[summary_prep_col].apply(remove_single_words)

    return df

In [ ]:
def random_sampling_by_percentage(dataframe, percentage):
    if not 0 <= percentage <= 100:
        raise ValueError("Percentage must be between 0 and 100")

    n = int(len(dataframe) * percentage / 100)
    return dataframe.sample(n=n, replace=False)

## Train Set

In [ ]:
# Train sampling

df_train_sample = random_sampling_by_percentage(df_train, 10)
df_train_sample.to_csv(os.path.join(dataset_path, 'df_train_sample_raw.csv'))

In [ ]:
df_train_sample = pd.read_csv(os.path.join(dataset_path, 'df_train_sample_raw.csv'), index_col=0)
df_train_sample.head()

,id,url,clean_article,clean_summary,extractive_summary
66916,130953,https://www.liputan6.com/news/read/130953/indonesia-60-detik,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela...",Pelabuhan Merak mulai dipadati ribuan pemudik tujuan Pulau Sumatra. Ribuan sapi milik warga di K...,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela..."
107704,188566,https://www.liputan6.com/news/read/188566/ramos-horta-mungkin-akan-mundur,"Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun...","Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembunuhan pada Februari lal...","Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun..."
111739,193009,https://www.liputan6.com/news/read/193009/sepuluh-masjid-rusak-akibat-gempa,"Liputan6.com, Palembang: Sedikitnya 10 masjid di Kecamatan Muara Payang dan Jarai, Lahat, Sumatr...",Warga khawatir melakukan salat di masjid karena takut tertimpa reruntuhan bangunan akibat gempa ...,"Warga khawatir tertimpa reruntuhan bangunan yang rusak berat. Diperkirakan, kerusakan akibat gem..."
176789,276762,https://www.liputan6.com/news/read/276762/perawat-tuntut-pengesahan-ruu-keperawatan,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d...",Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah dengan menuntut pe...,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d..."
164023,259411,https://www.liputan6.com/news/read/259411/pembagian-makanan-berakhir-ricuh,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa...",Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian bantuan makanan un...,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa..."


In [ ]:
df_train_prep = preprocessing(df_train_sample, 'clean_article', 'clean_summary', 'clean_article_prep', 'clean_summary_prep')
df_train_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
66916,130953,https://www.liputan6.com/news/read/130953/indonesia-60-detik,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela...",Pelabuhan Merak mulai dipadati ribuan pemudik tujuan Pulau Sumatra. Ribuan sapi milik warga di K...,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela...",pelabuhan merak mulai dipadati pemudik cilegon ribuan pemudik mulai memadati pelabuhan merak ban...,pelabuhan merak mulai dipadati ribuan pemudik tujuan pulau sumatra ribuan sapi milik warga di ka...
107704,188566,https://www.liputan6.com/news/read/188566/ramos-horta-mungkin-akan-mundur,"Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun...","Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembunuhan pada Februari lal...","Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun...",sydney presiden timor timur jose ramos horta yang terluka dalam percobaan pembunuhan pada februa...,presiden timor timur jose ramos horta yang terluka dalam percobaan pembunuhan pada februari lalu...
111739,193009,https://www.liputan6.com/news/read/193009/sepuluh-masjid-rusak-akibat-gempa,"Liputan6.com, Palembang: Sedikitnya 10 masjid di Kecamatan Muara Payang dan Jarai, Lahat, Sumatr...",Warga khawatir melakukan salat di masjid karena takut tertimpa reruntuhan bangunan akibat gempa ...,"Warga khawatir tertimpa reruntuhan bangunan yang rusak berat. Diperkirakan, kerusakan akibat gem...",palembang sedikitnya 10 masjid di kecamatan muara payang dan jarai lahat sumatra selatan yang ru...,warga khawatir melakukan salat di masjid karena takut tertimpa reruntuhan bangunan akibat gempa ...
176789,276762,https://www.liputan6.com/news/read/276762/perawat-tuntut-pengesahan-ruu-keperawatan,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d...",Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah dengan menuntut pe...,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d...",hari perawat sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah rabu di makassar s...,hari perawat sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah dengan menuntut pe...
164023,259411,https://www.liputan6.com/news/read/259411/pembagian-makanan-berakhir-ricuh,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa...",Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian bantuan makanan un...,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa...",port au prince ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian ban...,ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian bantuan makanan un...


In [ ]:
df_train_prep.to_csv(os.path.join(dataset_path, 'df_train_sample_prep.csv'))
df_train_prep = pd.read_csv(os.path.join(dataset_path, 'df_train_sample_prep.csv'), index_col=0)
df_train_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
66916,130953,https://www.liputan6.com/news/read/130953/indonesia-60-detik,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela...",Pelabuhan Merak mulai dipadati ribuan pemudik tujuan Pulau Sumatra. Ribuan sapi milik warga di K...,"Pelabuhan Merak Mulai Dipadati Pemudik Liputan6.com, Cilegon: Ribuan pemudik mulai memadati Pela...",pelabuhan merak mulai dipadati pemudik cilegon ribuan pemudik mulai memadati pelabuhan merak ban...,pelabuhan merak mulai dipadati ribuan pemudik tujuan pulau sumatra ribuan sapi milik warga di ka...
107704,188566,https://www.liputan6.com/news/read/188566/ramos-horta-mungkin-akan-mundur,"Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun...","Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembunuhan pada Februari lal...","Liputan6.com, Sydney: Presiden Timor Timur Jose Ramos-Horta, yang terluka dalam percobaan pembun...",sydney presiden timor timur jose ramos horta yang terluka dalam percobaan pembunuhan pada februa...,presiden timor timur jose ramos horta yang terluka dalam percobaan pembunuhan pada februari lalu...
111739,193009,https://www.liputan6.com/news/read/193009/sepuluh-masjid-rusak-akibat-gempa,"Liputan6.com, Palembang: Sedikitnya 10 masjid di Kecamatan Muara Payang dan Jarai, Lahat, Sumatr...",Warga khawatir melakukan salat di masjid karena takut tertimpa reruntuhan bangunan akibat gempa ...,"Warga khawatir tertimpa reruntuhan bangunan yang rusak berat. Diperkirakan, kerusakan akibat gem...",palembang sedikitnya 10 masjid di kecamatan muara payang dan jarai lahat sumatra selatan yang ru...,warga khawatir melakukan salat di masjid karena takut tertimpa reruntuhan bangunan akibat gempa ...
176789,276762,https://www.liputan6.com/news/read/276762/perawat-tuntut-pengesahan-ruu-keperawatan,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d...",Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah dengan menuntut pe...,"Liputan6.com, Makassar: Hari Perawat Sedunia diperingati sejumlah elemen mahasiswa di sejumlah d...",makassar hari perawat sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah rabu di m...,hari perawat sedunia diperingati sejumlah elemen mahasiswa di sejumlah daerah dengan menuntut pe...
164023,259411,https://www.liputan6.com/news/read/259411/pembagian-makanan-berakhir-ricuh,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa...",Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian bantuan makanan un...,"Liputan6.com, Port-au-Prince: Ribuan orang terlibat aksi dorong mendorong dengan tim relawan saa...",port au prince ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian ban...,ribuan orang terlibat aksi dorong mendorong dengan tim relawan saat pembagian bantuan makanan un...


## Validation Set

In [ ]:
df_val_prep = preprocessing(df_val, 'clean_article', 'clean_summary', 'clean_article_prep', 'clean_summary_prep')
df_val_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
0,1,https://www.liputan6.com/news/read/1/batas-penyerahan-aset-dua-pekan-lagi,"Liputan6.com, Jakarta: Pemerintah masih memberikan waktu dua minggu lagi kepada seluruh konglome...",Pemerintah memberikan tenggat 14 hari kepada para konglomerat penandatangan MSAA untuk menyerahk...,"Jika lewat dari tenggat tersebut, pemerintah akan menerapkan tindakan hukum. Sementara itu, peme...",pemerintah masih memberikan waktu dua minggu lagi kepada seluruh konglomerat yang telah menandat...,pemerintah memberikan tenggat 14 hari kepada para konglomerat penandatangan msaa untuk menyerahk...
1,2,https://www.liputan6.com/news/read/2/dpr-mengusulkan-resolusi-ipu-untuk-israel,"Liputan6.com, Jakarta: Kecaman demi kecaman keras ke Israel terus saja bergulir. Tindakan pemban...",MPR dan DPR mengutuk tindakan kekerasan tentara Israel terhadap warga Palestina. Hal itu akan di...,"Dalam orasinya, Akbar mengatakan bahwa DPR sangat mengutuk tindakan Israel terhadap bangsa Pales...",kecaman demi kecaman keras ke israel terus saja bergulir tindakan pembantaian terhadap palestina...,mpr dan dpr mengutuk tindakan kekerasan tentara israel terhadap warga palestina hal itu akan dic...
2,3,https://www.liputan6.com/news/read/3/dewi-wardah-meminta-jaminan-keamanan,"Liputan6.com, Jakarta: Janda mendiang Amir Biki, korban peristiwa Tanjungpriok Dewi Wardah, Juma...","Dewi Wardah, janda korban peristiwa Tanjungpriok meminta jaminan keamanan dari polisi. Ia mengak...","Liputan6.com, Jakarta: Janda mendiang Amir Biki, korban peristiwa Tanjungpriok Dewi Wardah, Juma...",janda mendiang amir biki korban peristiwa tanjungpriok dewi wardah jumat siang mendatangi kepoli...,dewi wardah janda korban peristiwa tanjungpriok meminta jaminan keamanan dari polisi ia mengaku ...
3,4,https://www.liputan6.com/news/read/4/kejati-jakarta-membantah-menerima-bap-tersangka,"Liputan6.com, Jakarta: Polisi telah menyerahkan tiga Berita acara pemeriksaan kasus pelemparan g...",Kapolda Metro Jaya mengaku telah menyerahkan BAP pelaku pengeboman Kedubes Malaysia ke Kejati DK...,"Anehnya, Kejati DKI Jakarta malah mengaku belum menerima BAP dari polisi. Beberapa hari silam, p...",polisi telah menyerahkan tiga berita acara pemeriksaan kasus pelemparan granat di kedutaan besar...,kapolda metro jaya mengaku telah menyerahkan bap pelaku pengeboman kedubes malaysia ke kejati dk...
6,7,https://www.liputan6.com/news/read/7/polisi-menembak-pencuri-tas-wisman,"Liputan6.com, Jakarta: Gara-gara berusaha kabur saat diminta menunjukkan barang hasil curian, Ro...",Seorang pencuri tas wisatawan asing ditembak polisi. Ia berusaha kabur saat diminta menunjukan h...,"Liputan6.com, Jakarta: Gara-gara berusaha kabur saat diminta menunjukkan barang hasil curian, Ro...",gara gara berusaha kabur saat diminta menunjukkan barang hasil curian rosihan bin usman tersangk...,seorang pencuri tas wisatawan asing ditembak polisi ia berusaha kabur saat diminta menunjukan ha...


In [ ]:
df_val_prep.to_csv(os.path.join(dataset_path, 'df_val_prep.csv'))
df_val_prep = pd.read_csv(os.path.join(dataset_path, 'df_val_prep.csv'), index_col=0)
df_val_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
0,1,https://www.liputan6.com/news/read/1/batas-penyerahan-aset-dua-pekan-lagi,"Liputan6.com, Jakarta: Pemerintah masih memberikan waktu dua minggu lagi kepada seluruh konglome...",Pemerintah memberikan tenggat 14 hari kepada para konglomerat penandatangan MSAA untuk menyerahk...,"Jika lewat dari tenggat tersebut, pemerintah akan menerapkan tindakan hukum. Sementara itu, peme...",pemerintah masih memberikan waktu dua minggu lagi kepada seluruh konglomerat yang telah menandat...,pemerintah memberikan tenggat 14 hari kepada para konglomerat penandatangan msaa untuk menyerahk...
1,2,https://www.liputan6.com/news/read/2/dpr-mengusulkan-resolusi-ipu-untuk-israel,"Liputan6.com, Jakarta: Kecaman demi kecaman keras ke Israel terus saja bergulir. Tindakan pemban...",MPR dan DPR mengutuk tindakan kekerasan tentara Israel terhadap warga Palestina. Hal itu akan di...,"Dalam orasinya, Akbar mengatakan bahwa DPR sangat mengutuk tindakan Israel terhadap bangsa Pales...",kecaman demi kecaman keras ke israel terus saja bergulir tindakan pembantaian terhadap palestina...,mpr dan dpr mengutuk tindakan kekerasan tentara israel terhadap warga palestina hal itu akan dic...
2,3,https://www.liputan6.com/news/read/3/dewi-wardah-meminta-jaminan-keamanan,"Liputan6.com, Jakarta: Janda mendiang Amir Biki, korban peristiwa Tanjungpriok Dewi Wardah, Juma...","Dewi Wardah, janda korban peristiwa Tanjungpriok meminta jaminan keamanan dari polisi. Ia mengak...","Liputan6.com, Jakarta: Janda mendiang Amir Biki, korban peristiwa Tanjungpriok Dewi Wardah, Juma...",janda mendiang amir biki korban peristiwa tanjungpriok dewi wardah jumat siang mendatangi kepoli...,dewi wardah janda korban peristiwa tanjungpriok meminta jaminan keamanan dari polisi ia mengaku ...
3,4,https://www.liputan6.com/news/read/4/kejati-jakarta-membantah-menerima-bap-tersangka,"Liputan6.com, Jakarta: Polisi telah menyerahkan tiga Berita acara pemeriksaan kasus pelemparan g...",Kapolda Metro Jaya mengaku telah menyerahkan BAP pelaku pengeboman Kedubes Malaysia ke Kejati DK...,"Anehnya, Kejati DKI Jakarta malah mengaku belum menerima BAP dari polisi. Beberapa hari silam, p...",polisi telah menyerahkan tiga berita acara pemeriksaan kasus pelemparan granat di kedutaan besar...,kapolda metro jaya mengaku telah menyerahkan bap pelaku pengeboman kedubes malaysia ke kejati dk...
6,7,https://www.liputan6.com/news/read/7/polisi-menembak-pencuri-tas-wisman,"Liputan6.com, Jakarta: Gara-gara berusaha kabur saat diminta menunjukkan barang hasil curian, Ro...",Seorang pencuri tas wisatawan asing ditembak polisi. Ia berusaha kabur saat diminta menunjukan h...,"Liputan6.com, Jakarta: Gara-gara berusaha kabur saat diminta menunjukkan barang hasil curian, Ro...",gara gara berusaha kabur saat diminta menunjukkan barang hasil curian rosihan bin usman tersangk...,seorang pencuri tas wisatawan asing ditembak polisi ia berusaha kabur saat diminta menunjukan ha...


## Test Set

In [ ]:
df_test_prep = preprocessing(df_test, 'clean_article', 'clean_summary', 'clean_article_prep', 'clean_summary_prep')
df_test_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
1,13020,https://www.liputan6.com/news/read/13020/bi-dinilai-masih-akan-dirundung-masalah,"Liputan6.com, Jakarta: Bank Indonesia dinilai masih akan menghadapi situasi sulit kendati Bank S...","Kendati Bank Sentral AS menurunkan suku bunganya, namun BI dinilai masih akan menemui masa sulit...","Liputan6.com, Jakarta: Bank Indonesia dinilai masih akan menghadapi situasi sulit kendati Bank S...",bank indonesia dinilai masih akan menghadapi situasi sulit kendati bank sentral amerika serikat ...,kendati bank sentral as menurunkan suku bunganya namun bi dinilai masih akan menemui masa sulit ...
3,13024,https://www.liputan6.com/news/read/13024/perubahan-kepmennaker-dinilai-dapat-mendorong-investor-...,"Liputan6.com, Jakarta: Penghapusan beberapa pasal menyangkut hak buruh dalam Keputusan Menteri T...","Revisi Kepmennaker Nomor 78 Tahun 2001, dinilai bukan semata-mata untuk membela kepentingan para...","Liputan6.com, Jakarta: Penghapusan beberapa pasal menyangkut hak buruh dalam Keputusan Menteri T...",penghapusan beberapa pasal menyangkut hak buruh dalam keputusan menteri tenaga kerja nomor 78 ta...,revisi kepmennaker nomor 78 tahun 2001 dinilai bukan semata mata untuk membela kepentingan para ...
4,13025,https://www.liputan6.com/news/read/13025/puluhan-pengunjung-diskotik-millenium-ditangkap,"Liputan6.com, Jakarta: Operasi Sadar Jaya yang dilancarkan Selasa (15/5) malam, sekitar pukul 23...",Polisi menangkap 32 pengunjung Diskotik Milenium karena tertangkap basah membawa pil ekstasi. Pe...,"Dari operasi tersebut, polisi menangkap 32 pengunjung diskotik yang tertangkap basah membawa 66 ...",operasi sadar jaya yang dilancarkan selasa malam sekitar pukul 2300 wib mengejutkan pengunjung d...,polisi menangkap 32 pengunjung diskotik milenium karena tertangkap basah membawa pil ekstasi pen...
6,13027,https://www.liputan6.com/news/read/13027/ruu-penyiaran-memerlukan-perubahan,"Liputan6.com, Jakarta: Sejumlah pasal dalam Rancangan Undang-undang Penyiaran mendapat sorotan t...",Praktisi penyiaran menyoroti sejumlah pasal dalam RUU Penyiaran tak memihak kepada kepentingan r...,"RUU tersebut dipandang tak berpihak pada kepentingan rakyat. Ichlasul juga menganggap, Pasal 26 ...",sejumlah pasal dalam rancangan undang undang penyiaran mendapat sorotan tajam dari para praktisi...,praktisi penyiaran menyoroti sejumlah pasal dalam ruu penyiaran tak memihak kepada kepentingan r...
8,13030,https://www.liputan6.com/news/read/13030/pusat-kerajinan-kasongan-kian-telantar,"Liputan6.com, Bantul: Pusat desa kerajinan atau Pasar Kriya Desa Kasongan, Bantul, Yogyakarta, s...","Karena ditinggalkan para perajin, kondisi Desa Kasongan sebagai pusat kerajinan keramik makin te...",Sejumlah benda antik yang dianggap menjadi cikal bakal perkembangan seni kerajinan gerabah dan k...,bantul pusat desa kerajinan atau pasar kriya desa kasongan bantul yogyakarta sebelumnya dibangun...,karena ditinggalkan para perajin kondisi desa kasongan sebagai pusat kerajinan keramik makin tel...


In [ ]:
df_test_prep.to_csv(os.path.join(dataset_path, 'df_test_prep.csv'))
df_test_prep = pd.read_csv(os.path.join(dataset_path, 'df_test_prep.csv'), index_col=0)
df_test_prep.head()

,id,url,clean_article,clean_summary,extractive_summary,clean_article_prep,clean_summary_prep
1,13020,https://www.liputan6.com/news/read/13020/bi-dinilai-masih-akan-dirundung-masalah,"Liputan6.com, Jakarta: Bank Indonesia dinilai masih akan menghadapi situasi sulit kendati Bank S...","Kendati Bank Sentral AS menurunkan suku bunganya, namun BI dinilai masih akan menemui masa sulit...","Liputan6.com, Jakarta: Bank Indonesia dinilai masih akan menghadapi situasi sulit kendati Bank S...",bank indonesia dinilai masih akan menghadapi situasi sulit kendati bank sentral amerika serikat ...,kendati bank sentral as menurunkan suku bunganya namun bi dinilai masih akan menemui masa sulit ...
3,13024,https://www.liputan6.com/news/read/13024/perubahan-kepmennaker-dinilai-dapat-mendorong-investor-...,"Liputan6.com, Jakarta: Penghapusan beberapa pasal menyangkut hak buruh dalam Keputusan Menteri T...","Revisi Kepmennaker Nomor 78 Tahun 2001, dinilai bukan semata-mata untuk membela kepentingan para...","Liputan6.com, Jakarta: Penghapusan beberapa pasal menyangkut hak buruh dalam Keputusan Menteri T...",penghapusan beberapa pasal menyangkut hak buruh dalam keputusan menteri tenaga kerja nomor 78 ta...,revisi kepmennaker nomor 78 tahun 2001 dinilai bukan semata mata untuk membela kepentingan para ...
4,13025,https://www.liputan6.com/news/read/13025/puluhan-pengunjung-diskotik-millenium-ditangkap,"Liputan6.com, Jakarta: Operasi Sadar Jaya yang dilancarkan Selasa (15/5) malam, sekitar pukul 23...",Polisi menangkap 32 pengunjung Diskotik Milenium karena tertangkap basah membawa pil ekstasi. Pe...,"Dari operasi tersebut, polisi menangkap 32 pengunjung diskotik yang tertangkap basah membawa 66 ...",operasi sadar jaya yang dilancarkan selasa malam sekitar pukul 2300 wib mengejutkan pengunjung d...,polisi menangkap 32 pengunjung diskotik milenium karena tertangkap basah membawa pil ekstasi pen...
6,13027,https://www.liputan6.com/news/read/13027/ruu-penyiaran-memerlukan-perubahan,"Liputan6.com, Jakarta: Sejumlah pasal dalam Rancangan Undang-undang Penyiaran mendapat sorotan t...",Praktisi penyiaran menyoroti sejumlah pasal dalam RUU Penyiaran tak memihak kepada kepentingan r...,"RUU tersebut dipandang tak berpihak pada kepentingan rakyat. Ichlasul juga menganggap, Pasal 26 ...",sejumlah pasal dalam rancangan undang undang penyiaran mendapat sorotan tajam dari para praktisi...,praktisi penyiaran menyoroti sejumlah pasal dalam ruu penyiaran tak memihak kepada kepentingan r...
8,13030,https://www.liputan6.com/news/read/13030/pusat-kerajinan-kasongan-kian-telantar,"Liputan6.com, Bantul: Pusat desa kerajinan atau Pasar Kriya Desa Kasongan, Bantul, Yogyakarta, s...","Karena ditinggalkan para perajin, kondisi Desa Kasongan sebagai pusat kerajinan keramik makin te...",Sejumlah benda antik yang dianggap menjadi cikal bakal perkembangan seni kerajinan gerabah dan k...,bantul pusat desa kerajinan atau pasar kriya desa kasongan bantul yogyakarta sebelumnya dibangun...,karena ditinggalkan para perajin kondisi desa kasongan sebagai pusat kerajinan keramik makin tel...


# EDA

In [ ]:
df_train['location'] = df_train['clean_article_wp'].apply(get_first_word) #get first word (usually the location value)

## Top 10 Cities

In [ ]:
count_location = df_train_prep[['id','location']].groupby('location').count().sort_values('id', ascending=False).head(70).reset_index()
count_location.rename(columns={'id':'count'}, inplace=True)
count_location.loc[count_location['location']=='New', 'location']='New York'
count_location.loc[count_location['location']=='Los', 'location']='Los Angeles'
count_location.loc[count_location['location']=='Kuala', 'location']='Kuala Lumpur'
count_location.head(15)

,location,count
0,Jakarta,62906
1,Surabaya,3442
2,Bandung,3044
3,Semarang,2977
4,Medan,2857
5,Makassar,2204
6,Yogyakarta,2123
7,Denpasar,2110
8,Tangerang,1948
9,Bogor,1926


## Most Used Words - No Stopwords

In [ ]:
text = list(df_train_prep['clean_article_no_sw'])
from collections import Counter

kosa_kata = []
for line in text:
    kosa_kata.extend(line.split())

word_counts = Counter(kosa_kata)
sorted_word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))

sorted_word_counts_df = pd.DataFrame(list(sorted_word_counts.items()), columns=['word', 'count'])
sorted_word_counts_df.head(20)

,word,count
0,jakarta,167669
1,warga,125601
2,rumah,96208
3,baca,85849
4,polisi,83049
5,indonesia,79099
6,tim,74936
7,korban,72493
8,orang,72010
9,rp,70217


## Bigram Analysis

In [ ]:
text = list(df_train_prep['clean_article_no_sw'])
bi_grams = []

for line in text:
    words = line.split()
    line_bigrams = list(bigrams(words))
    bi_grams.extend(line_bigrams)

bigram_counts = Counter(bi_grams)
sorted_bigram_counts = dict(sorted(bigram_counts.items(), key=lambda item: item[1], reverse=True))
sorted_bigram_counts_df = pd.DataFrame(list(sorted_bigram_counts.items()), columns=['bigram', 'count'])
sorted_bigram_counts_df

,bigram,count
0,"(rumah, sakit)",27146
1,"(jawa, barat)",23482
2,"(jawa, timur)",20374
3,"(jakarta, pusat)",13174
4,"(anak, anak)",12388
...,...,...
7384153,"(direformasi, langsung)",1
7384154,"(beatles, menonton)",1
7384155,"(menonton, paul)",1
7384156,"(mccartney, manggung)",1


In [ ]:
sorted_bigram_counts_df.head(20)

,bigram,count
0,"(rumah, sakit)",27146
1,"(jawa, barat)",23482
2,"(jawa, timur)",20374
3,"(jakarta, pusat)",13174
4,"(anak, anak)",12388
5,"(jakarta, selatan)",11664
6,"(amerika, serikat)",10858
7,"(susilo, bambang)",9847
8,"(bambang, yudhoyono)",9672
9,"(kepolisian, resor)",9533


## Trigram Analysis

In [ ]:
text = list(df_train_prep['clean_article_no_sw'])
trigrams_list = []

for line in text:
    words = line.split()
    line_trigrams = list(trigrams(words))
    trigrams_list.extend(line_trigrams)  # Use trigrams_list instead of trigrams

trigram_counts = Counter(trigrams_list)
sorted_trigram_counts = dict(sorted(trigram_counts.items(), key=lambda item: item[1], reverse=True))
sorted_trigram_counts_df = pd.DataFrame(list(sorted_trigram_counts.items()), columns=['trigram', 'count'])
sorted_trigram_counts_df

,trigram,count
0,"(susilo, bambang, yudhoyono)",9644
1,"(presiden, susilo, bambang)",6948
2,"(nanggroe, aceh, darussalam)",5364
3,"(dirawat, rumah, sakit)",5041
4,"(bahan, bakar, minyak)",4713
...,...,...
16090515,"(menonton, paul, mccartney)",1
16090516,"(paul, mccartney, manggung)",1
16090517,"(mccartney, manggung, menonton)",1
16090518,"(manggung, menonton, the)",1


In [ ]:
sorted_trigram_counts_df.head(20)

,trigram,count
0,"(susilo, bambang, yudhoyono)",9644
1,"(presiden, susilo, bambang)",6948
2,"(nanggroe, aceh, darussalam)",5364
3,"(dirawat, rumah, sakit)",5041
4,"(bahan, bakar, minyak)",4713
5,"(partai, demokrasi, indonesia)",4565
6,"(demokrasi, indonesia, perjuangan)",4463
7,"(bandung, jawa, barat)",4413
8,"(surabaya, jawa, timur)",4192
9,"(bogor, jawa, barat)",3463


# Transfer Learning

In [ ]:
import datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train_prep)
val_dataset = Dataset.from_pandas(df_val_prep)
test_dataset = Dataset.from_pandas(df_test_prep)

In [ ]:
train_dataset

Dataset({
    features: ['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary', 'clean_article_prep', 'clean_summary_prep', '__index_level_0__'],
    num_rows: 19388
})

In [ ]:
val_dataset

Dataset({
    features: ['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary', 'clean_article_prep', 'clean_summary_prep', '__index_level_0__'],
    num_rows: 4948
})

In [ ]:
test_dataset

Dataset({
    features: ['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary', 'clean_article_prep', 'clean_summary_prep', '__index_level_0__'],
    num_rows: 3862
})

## Modelling

In [21]:
batch_size=4
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [ ]:
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["clean_article_prep"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["clean_summary_prep"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
# train_data = train_data.select(range(32))

train_data = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=['id',
                    'url',
                    'clean_article',
                    'clean_summary',
                    'extractive_summary',
                    'clean_article_prep',
                    '__index_level_0__']
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/19388 [00:00<?, ? examples/s]

In [ ]:
val_data = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=['id',
                    'url',
                    'clean_article',
                    'clean_summary',
                    'extractive_summary',
                    'clean_article_prep',
                    '__index_level_0__']
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/4948 [00:00<?, ? examples/s]

In [ ]:
test_data = test_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=['id',
                    'url',
                    'clean_article',
                    'clean_summary',
                    'extractive_summary',
                    'clean_article_prep',
                    '__index_level_0__']
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/3862 [00:00<?, ? examples/s]

In [ ]:
# train_data.save_to_disk(os.path.join(dataset_path, 'train_data.arrow'))
# val_data.save_to_disk(os.path.join(dataset_path, 'val_data.arrow'))
# test_data.save_to_disk(os.path.join(dataset_path, 'test_data.arrow'))

Saving the dataset (0/1 shards):   0%|          | 0/19388 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4948 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3862 [00:00<?, ? examples/s]

In [20]:
train_data = load_from_disk(os.path.join(dataset_path, 'train_data.arrow'))
val_data = load_from_disk(os.path.join(dataset_path, 'val_data.arrow'))
test_data = load_from_disk(os.path.join(dataset_path, 'test_data.arrow'))

### Cahya Bert2Bert

In [22]:
# set special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [23]:
# Freeze encoder layer
for param in model.decoder.parameters():
    param.requires_grad = False

In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=2000,
    save_steps=5000,
    warmup_steps=1000,
    num_train_epochs=3,
    max_steps=10000,
    logging_dir="./logs",
    learning_rate=1e-5
)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=train_data,
  eval_dataset=val_data,
  tokenizer=tokenizer,
  data_collator=data_collator
  )

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warn

Epoch,Training Loss,Validation Loss
1,1.407400,3.491886


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder

TrainOutput(global_step=10000, training_loss=1.51428525390625, metrics={'train_runtime': 1882.9267, 'train_samples_per_second': 10.622, 'train_steps_per_second': 5.311, 'total_flos': 1.226917281792e+16, 'train_loss': 1.51428525390625, 'epoch': 1.03})

In [26]:
trainer.save_model(os.path.join(model_path, 'model_batch_2_lr_1e-5'))

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
